In [520]:
import math 
import random

class Node():
    def __init__(self, value, _backward=None, parents=[]):
        self.value = value
        self.grad = 0
        self._backward = _backward
        self.parents = parents
    
    def __add__(self, other):
        other = other if isinstance(other, Node) else Node(other)
        value_add = self.value + other.value

        def _backward():
            self.grad = 1 # c = a + b => dc/da = 1
            other.grad = 1 # c = a + b => dc/db = 1
        return Node(value_add, parents=[self, other], _backward=_backward)

    def __radd__(self,other):
        return self + other

    def __sub__(self, other):
        other = other if isinstance(other, Node) else Node(other)
        value_sub = self.value - other.value

        def _backward():
            self.grad = 1 # c = a + b => dc/da = 1
            other.grad = -1 # c = a + b => dc/db = 1
        return Node(value_sub, parents=[self, other], _backward=_backward)
    
    def __rsub__(self,other):
        return self - other

    def __mul__(self, other):
        other = other if isinstance(other, Node) else Node(other)
        value_mul = self.value * other.value

        def _backward():
            self.grad = other.value # c = a * b => dc/da = b
            other.grad = self.value # c = a * b => dc/db = a
        
        return Node(value_mul, parents=[self, other], _backward=_backward)
    
    def __rmul__(self,other):
        return self * other

    def __pow__(self, other):
        value_pow = self.value ** other

        def _backward():
            self.grad = other * value_pow ** (other - 1)
        
        return Node(value_pow, parents=[self,], _backward=_backward)

    def sigmoid(self):
        value_sigmoid = 1 / (1 + math.exp(-self.value))
        
        def _backward():
            self.grad += value_sigmoid * (1 - value_sigmoid)
    
        return Node(value_sigmoid, parents=[self], _backward=_backward)

    def __repr__(self):
        return f"Node( value={self.value}, grad={self.grad})"

    # BFS algorithm to perform backpropagation
    def backward(self):
        self.local_grad = 1
        self.global_grad = 1

        visited = [self]
        queue = [self]

        while queue != []:
            node = queue.pop(0)        
            for parent in node.parents:
                node._backward()
                visited.append(parent)
                queue.append(parent)


In [521]:
class Neuron():
    def __init__(self, n_weights):
        self.weights = [Node(random.random()) for _ in range(n_weights)]
        self.bias = Node(random.random())
        self.parameters = self.weights + [self.bias]
    
    def __call__(self, data):
        sum_value = sum( (w*x for w,x in zip(self.weights, data)), self.bias)
        return sum_value.sigmoid()

In [522]:
# Fixed data set
n_samples = 10

x_0 = [random.random() for _ in range(n_samples)]
x_1 = [100*(0.5 - random.random()) for _ in range(n_samples)]
x = list(zip(x_0, x_1))

y_true = [1 if i[1]>0 else 0 for i in x ]

In [523]:
n.parameters

[Node( value=0.408417344694878, grad=0),
 Node( value=0.9751528210876826, grad=0),
 Node( value=0.7635167197009194, grad=0)]

In [524]:
n = Neuron(2)

In [525]:
n(x[0])

Node( value=3.0032674936280976e-09, grad=0)

In [531]:
n_epoch = 1000
learning_rate = 0.005

for i in range(n_epoch):
    # forward pass
    y_pred = [n(x) for x in x]
    y_loss = sum((y_pred_i - y_true_i)**2 for y_pred_i,y_true_i in zip(y_pred, y_true))*(1/n_samples)

    # backward
    for p in n.parameters:
        p.grad = 0
    y_loss.backward()

    # update
    for p in n.parameters:
        p.value += -learning_rate*p.grad*(1/n_samples)

    if i%100 == 0:
        print(y_loss.value)


0.008135064362473993
0.004154670877086152
0.0019904369675540533
0.0009089471752828902
0.0004011049179332367
0.00017287091950747785
7.333031895573574e-05
3.078049382349038e-05
1.2831436590032965e-05
5.32512229672087e-06


In [532]:
y_true

[0, 1, 0, 1, 0, 0, 1, 1, 1, 1]

In [533]:
y_pred

[Node( value=1.0083689353690345e-146, grad=1),
 Node( value=1.0, grad=1),
 Node( value=3.234656072762598e-105, grad=1),
 Node( value=1.0, grad=1),
 Node( value=9.534646346637487e-163, grad=1),
 Node( value=4.1408771624236905e-76, grad=1),
 Node( value=1.0, grad=1),
 Node( value=1.0, grad=1),
 Node( value=1.0, grad=1),
 Node( value=0.9952850092005565, grad=1)]